In [1]:
#%% Setting Up
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import xarray as xr
import xesmf as xe
import networkx as nx
import rioxarray as rxr

import geopandas as gpd
import seaborn as sns
import matplotlib.pyplot as plt

from shapely.geometry import Point
from shapely.geometry import Polygon

import glob
import os
import itertools
import tqdm
import gc
import time
import pickle

from joblib import Parallel, delayed

import rioxarray as rxr

import configparser
cfg = configparser.ConfigParser()
cfg.optionxform = str
cfg.read('/home/sarth/rootdir/datadir/assets/defaults.ini')
cfg = {s: dict(cfg.items(s)) for s in cfg.sections()}
PATHS = cfg['PATHS']

print("Setting up...")

Setting up...


In [2]:
#%% Region-Specific: CAMELS-US
DIRNAME = '03min_GloFAS_CAMELS-US'
SAVE_PATH = os.path.join(PATHS['devp_datasets'], DIRNAME)
resolution = 0.05
lon_360_180 = lambda x: (x + 180) % 360 - 180 # convert 0-360 to -180-180
lon_180_360 = lambda x: x % 360 # convert -180-180 to 0-360
region_bounds = {
    'minx': -130,
    'miny': 20,
    'maxx': -65,
    'maxy': 50
}

# camels_attributes_graph = pd.read_csv(os.path.join(SAVE_PATH, 'graph_attributes.csv'), index_col=0)
# camels_attributes_graph.index = camels_attributes_graph.index.map(lambda x: str(x).zfill(8))
# camels_attributes_graph['huc_02'] = camels_attributes_graph['huc_02'].map(lambda x: str(x).zfill(2))
# camels_graph = camels_attributes_graph.copy()
# camels_graph = camels_graph[camels_graph['area_percent_difference'] < 10]
# camels_graph = camels_graph[camels_graph['num_nodes'] > 1]
# print(f"Number of CAMELS-US catmt's: {len(camels_graph)}")
# del camels_attributes_graph

camels_graph = pd.read_csv(os.path.join(SAVE_PATH, 'nested_gauges', 'graph_attributes_with_nesting.csv'), index_col=0)
camels_graph.index = camels_graph.index.map(lambda x: str(x).zfill(8))
camels_graph['huc_02'] = camels_graph['huc_02'].map(lambda x: str(x).zfill(2))
# camels_graph = camels_graph[camels_graph['nesting'].isin(['not_nested', 'nested_downstream'])]
# camels_graph = camels_graph.reset_index()
print(f"Number of catmt's with nesting: {len(camels_graph)}")

Number of catmt's with nesting: 395


In [3]:
def idx_to_map(ds, var_name):
    lats = ds.lat.values
    lons = ds.lon.values
    catmt_var_map = xr.DataArray(
        np.zeros((len(lats), len(lons)), dtype = np.float32)*np.nan,
        dims = ['lat', 'lon'],
        coords = {'lat': lats, 'lon': lons}
    )
    for idx in ds.idx.values:
        lat, lon = ds['idx2lat'].sel(idx = idx).values, ds['idx2lon'].sel(idx = idx).values
        catmt_var_map.loc[dict(lat = lat, lon = lon)] = ds[var_name].sel(idx = idx).values
    return catmt_var_map

- dynamic
    - Daymet (Done)
    - ECMWF_HRES (Done)
    - ERA5 (Done)
    - ERA5-Land (Done)
    - GLEAM4 (Done)
    - GloFAS (Done)
    - GPM (Done)
- static
    - ERA5 (Done)
    - GloFAS (Done)
    - HWSD (Done)
    - MERIT-Hydro (Done)
- solar insolation (Done)
- spatial encodings (Done)
- time encodings (Done)
- uparea (Done)
- USGS (Done)

Train: 1999 - 2009 (10 WYs)
Validation: 2009 - 2016 (07 WYs)
Test: 2016 - 2022 (06 WYs)


1999 - 2022 (24 yrs)
2016 - 2019 (04 yrs)

In [4]:
START_DATE = pd.Timestamp('1998-01-01')
END_DATE = pd.Timestamp('2022-12-31')

In [5]:
# camels_graph_reversed = camels_graph.sort_values(by='area_geospa_fabric', ascending=True)
camels_graph = camels_graph[camels_graph['huc_02'] != '05']
camels_graph

,huc_02,gauge_lon,gauge_lat,area_geospa_fabric,snapped_lon,snapped_lat,snapped_uparea,snapped_iou,area_percent_difference,num_nodes,num_edges,nesting
gauge_id,,,,,,,,,,,,
06452000,10,-99.55649,43.74833,25817.78,-99.525,43.775,26081.820000,0.942874,1.022710,1158.0,1157.0,nested_downstream
13340000,17,-116.25750,46.47833,14270.76,-116.275,46.475,14113.031000,0.940266,1.105257,657.0,656.0,nested_downstream
06447000,10,-101.52487,43.75250,12869.46,-101.575,43.725,12933.430000,0.929852,0.497066,573.0,572.0,nested_upstream
06360500,10,-100.84292,45.25582,12601.47,-100.875,45.275,12741.733000,0.926271,1.113074,584.0,583.0,not_nested
06354000,10,-100.93444,46.37611,10626.74,-100.925,46.375,10708.444000,0.919406,0.768854,500.0,499.0,nested_downstream
...,...,...,...,...,...,...,...,...,...,...,...,...
10310500,16,-119.79824,39.11324,47.65,-119.825,39.125,48.000664,0.621974,0.735912,2.0,1.0,not_nested
01350140,02,-74.47293,42.42897,47.57,-74.525,42.425,45.708480,0.291629,3.913220,2.0,1.0,not_nested
08271000,13,-105.53084,36.50836,46.39,-105.475,36.525,49.693077,0.571310,7.120237,2.0,1.0,not_nested


# Init Inventory

In [6]:
def init_zarr(huc = '10', gauge_id = '06452000', res_arcmins = 3, round_up_places = 3, uparea = None):
    res_degrees = res_arcmins / 60
    nodes_coords = pd.read_csv(os.path.join(SAVE_PATH, 'graph_files', huc, gauge_id, 'nodes_coords.csv'), index_col = 0)
    edges = pd.read_csv(os.path.join(SAVE_PATH, 'graph_files', huc, gauge_id, 'edges.csv'), index_col = 0)
    edges_idx = edges.loc[:,['from_idx', 'to_idx']].values.tolist()

    lats = np.arange(max(nodes_coords['lat']), min(nodes_coords['lat']) - res_degrees / 2, -res_degrees).astype(np.float32)
    lats = [round(x, round_up_places) for x in lats]

    lons = np.arange(min(nodes_coords['lon']), max(nodes_coords['lon']) + res_degrees / 2, res_degrees).astype(np.float32)
    lons = [round(x, round_up_places) for x in lons]

    mask = xr.DataArray(np.zeros((len(lats), len(lons)), dtype = bool), coords = [lats, lons], dims = ['lat', 'lon'])
    for _, row in nodes_coords.iterrows():
        mask.loc[row['lat'], row['lon']] = True

    catmt = xr.Dataset()
    catmt['mask'] = mask

    catmt.attrs['watershed'] = f"CAMELS-US/HUC:{huc}/GaugeID:{gauge_id}"
    catmt.attrs['gauge_idx'] = [0]
    catmt.attrs['resolution(degrees)'] = res_degrees
    catmt.attrs['resolution(arcmins)'] = res_arcmins
    catmt.attrs['edges_idx'] = edges_idx
    catmt.attrs['drainage_area_km2'] = uparea

    catmt['idx2lat'] = xr.DataArray(nodes_coords['lat'].values.astype(np.float32), dims = 'idx')
    catmt['idx2lon'] = xr.DataArray(nodes_coords['lon'].values.astype(np.float32), dims='idx')
    catmt = catmt.assign_coords(idx = nodes_coords.index.values.astype(np.int32))

    catmt = catmt.assign_coords(time = pd.date_range(start = '1998-01-01', end = '2022-12-31', freq = 'D'))
    catmt = catmt.sel(time=~((catmt.time.dt.month == 2) & (catmt.time.dt.day == 29)))

    catmt = catmt.assign_coords(lead = np.arange(1, 10+1, dtype = np.int32))

    os.makedirs(os.path.join(SAVE_PATH, 'inventory', huc), exist_ok = True)
    catmt.to_zarr(os.path.join(SAVE_PATH, 'inventory', huc, f'{gauge_id}.zarr'), mode = 'w')

for idx, row in tqdm.tqdm(camels_graph.iterrows(), total=len(camels_graph)):
    huc, gauge_id = row['huc_02'], row.name
    uparea = row['area_geospa_fabric']
    init_zarr(huc, gauge_id, res_arcmins = 3, round_up_places = 3, uparea = uparea)

  0%|          | 0/369 [00:00<?, ?it/s]

100%|██████████| 369/369 [00:44<00:00,  8.33it/s]


# ERA5

In [7]:
def compile_ERA5_zarr(huc, gauge_id):

    warnings.filterwarnings('ignore')

    data_START_DATE = pd.Timestamp('1998-01-01')
    data_END_DATE = pd.Timestamp('2020-12-31')

    catmt = xr.open_zarr(os.path.join(SAVE_PATH, 'inventory', huc, f'{gauge_id}.zarr'))
    var_names = [
        'total_precipitation',
        'surface_net_solar_radiation',
        'surface_net_thermal_radiation',
        'evaporation',
        'potential_evaporation',
        'runoff',
        'surface_runoff',
        'sub_surface_runoff'
    ]
    for var_name in var_names:
        catmt[f'dynamic_ERA5_{var_name}'] = xr.DataArray(
            np.zeros((len(catmt.time), len(catmt.idx)), dtype = np.float32)*np.nan, 
            dims = ['time', 'idx'], 
            coords = {'time': catmt.time, 'idx': catmt.idx}
        )
        data = pd.read_csv(os.path.join(SAVE_PATH, 'graph_features', huc, gauge_id, 'dynamic', 'ERA5', f"{var_name}.csv"), index_col = 0, parse_dates = True)
        data = data.loc[data_START_DATE:data_END_DATE]
        data = data * 24
        for idx in data.columns:
            catmt[f'dynamic_ERA5_{var_name}'].loc[dict(time=slice(data_START_DATE, data_END_DATE), idx=int(idx))] = data[idx].values
        del data, idx
    
    var_names = [
        '2m_temperature',
        'surface_pressure',
        '2m_dewpoint_temperature',
        '10m_u_component_of_wind',
        '10m_v_component_of_wind',
        'snowfall',
        'snow_depth',
        'snowmelt',
        'volumetric_soil_water_layer_1',
        'volumetric_soil_water_layer_2',
        'volumetric_soil_water_layer_3',
        'volumetric_soil_water_layer_4'
    ]
    for var_name in var_names:
        catmt[f'dynamic_ERA5_{var_name}'] = xr.DataArray(
            np.zeros((len(catmt.time), len(catmt.idx)), dtype = np.float32)*np.nan, 
            dims = ['time', 'idx'], 
            coords = {'time': catmt.time, 'idx': catmt.idx}
        )
        data = pd.read_csv(os.path.join(SAVE_PATH, 'graph_features', huc, gauge_id, 'dynamic', 'ERA5', f"{var_name}.csv"), index_col = 0, parse_dates = True)
        data = data.loc[data_START_DATE:data_END_DATE]
        for idx in data.columns:
            catmt[f'dynamic_ERA5_{var_name}'].loc[dict(time=slice(data_START_DATE, data_END_DATE), idx=int(idx))] = data[idx].values
        del data, idx

    catmt.to_zarr(os.path.join(SAVE_PATH, 'inventory', huc, f'{gauge_id}.zarr'), mode = 'a')

# for idx, row in tqdm.tqdm(camels_graph.iterrows(), total=len(camels_graph)):
#     huc, gauge_id = row['huc_02'], row.name
#     compile_ERA5_zarr(huc, gauge_id)

with Parallel(n_jobs=8, verbose=10) as parallel:
    parallel(
        delayed(compile_ERA5_zarr)(row['huc_02'], row.name) for idx, row in tqdm.tqdm(camels_graph.iterrows(), total=len(camels_graph))
    )

  0%|          | 0/369 [00:00<?, ?it/s][Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.


  4%|▍         | 16/369 [00:29<12:48,  2.18s/it][Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:   43.5s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:  1.6min
100%|██████████| 369/369 [09:58<00:00,  1.62s/it]
[Parallel(n_jobs=8)]: Done 369 out of 369 | elapsed: 10.0min finished


In [ ]:
def compile_ERA5_static_continuous_zarr(huc, gauge_id):
    catmt = xr.open_zarr(os.path.join(SAVE_PATH, 'inventory', huc, f'{gauge_id}.zarr'))
    var_names = ['high_vegetation_cover','low_vegetation_cover']
    for var_name in var_names:
        catmt[f'static_ERA5_{var_name}'] = xr.DataArray(
            np.zeros((len(catmt.idx)), dtype = np.float32)*np.nan, 
            dims = ['idx'], 
            coords = {'idx': catmt.idx}
        )
        data = pd.read_csv(os.path.join(SAVE_PATH, 'graph_features', huc, gauge_id, 'static', 'ERA5', f"static_{var_name}.csv"), index_col = 0, parse_dates = True)
        for idx in data.columns:
            catmt[f'static_ERA5_{var_name}'].loc[dict(idx = int(idx))] = data[idx].values[0]
        del data, idx
    catmt.to_zarr(os.path.join(SAVE_PATH, 'inventory', huc, f'{gauge_id}.zarr'), mode = 'a')

# for idx, row in tqdm.tqdm(camels_graph.iterrows(), total=len(camels_graph)):
#     huc, gauge_id = row['huc_02'], row.name
#     compile_ERA5_static_continuous_zarr(huc, gauge_id)

with Parallel(n_jobs=8, verbose=10) as parallel:
    parallel(
        delayed(compile_ERA5_static_continuous_zarr)(row['huc_02'], row.name) for idx, row in tqdm.tqdm(camels_graph.iterrows(), total=len(camels_graph))
    )

In [ ]:
# Categorical variables
def compile_ERA5_static_categorical_zarr(huc, gauge_id):
    catmt = xr.open_zarr(os.path.join(SAVE_PATH, 'inventory', huc, f'{gauge_id}.zarr'))
    var_names = ['soil_type', 'type_of_high_vegetation', 'type_of_low_vegetation']
    var_classes = [
        [0,1,2,3,4,5,6,7],
        [0,3,4,5,6,18,19],
        [0,1,2,7,9,10,11,13,16,17,20]
    ]
    var_num_classes = [len(x) for x in var_classes]
    for var_name, var_class in zip(var_names, var_classes):
        # Store as a string, flip the bit to get the class
        catmt[f'static_ERA5_{var_name}'] = xr.DataArray(
            np.zeros((len(catmt.idx)), dtype = int),
            dims = ['idx'], 
            coords = {'idx': catmt.idx}
        )
        data = pd.read_csv(os.path.join(SAVE_PATH, 'graph_features', huc, gauge_id, 'static', 'ERA5', f"static_{var_name}.csv"), index_col = 0, parse_dates = True)
        for idx in data.columns:
            node_class = data[idx].values[0]
            node_class = node_class if node_class in var_class else var_class[0]
            # node_class_idx = np.where(np.isin(var_class, node_class))[0][0]
            # node_class_bool = np.zeros(len(var_class), dtype = np.int32)
            # node_class_bool[node_class_idx] = 1
            # node_class_bool_str = ''.join([str(x) for x in node_class_bool])
            # catmt[f'static_ERA5_{var_name}'].loc[dict(idx = int(idx))] = node_class_bool_str
            catmt[f'static_ERA5_{var_name}'].loc[dict(idx = int(idx))] = int(node_class)
        del data, idx
    catmt.to_zarr(os.path.join(SAVE_PATH, 'inventory', huc, f'{gauge_id}.zarr'), mode = 'a')

# for idx, row in tqdm.tqdm(camels_graph.iterrows(), total=len(camels_graph)):
#     huc, gauge_id = row['huc_02'], row.name
#     compile_ERA5_static_categorical_zarr(huc, gauge_id)

with Parallel(n_jobs=8, verbose=10) as parallel:
    parallel(
        delayed(compile_ERA5_static_categorical_zarr)(row['huc_02'], row.name) for idx, row in tqdm.tqdm(camels_graph.iterrows(), total=len(camels_graph))
    )

# GLEAM4

In [10]:
def compile_GLEAM4_zarr(huc, gauge_id):
    warnings.filterwarnings('ignore')

    data_START_DATE = pd.Timestamp('1998-01-01')
    data_END_DATE = pd.Timestamp('2020-12-31')

    catmt = xr.open_zarr(os.path.join(SAVE_PATH, 'inventory', huc, f'{gauge_id}.zarr'))
    var_names = ['Ep', 'SMrz', 'SMs'] # ['Ep', 'SMrz', 'SMs', 'Eb', 'Ei', 'Es', 'Et', 'Ew', 'S', 'H']
    for var_name in var_names:
        catmt[f'dynamic_GLEAM4_{var_name}'] = xr.DataArray(
            np.zeros((len(catmt.time), len(catmt.idx)), dtype = np.float32)*np.nan, 
            dims = ['time', 'idx'], 
            coords = {'time': catmt.time, 'idx': catmt.idx}
        )
        data = pd.read_csv(os.path.join(SAVE_PATH, 'graph_features', huc, gauge_id, 'dynamic', 'GLEAM4', f"{var_name}.csv"), index_col = 0, parse_dates = True)
        data = data.loc[data_START_DATE:data_END_DATE]
        for idx in data.columns:
            catmt[f'dynamic_GLEAM4_{var_name}'].loc[dict(time=slice(data_START_DATE, data_END_DATE), idx=int(idx))] = data[idx].values
        del data, idx

    catmt.to_zarr(os.path.join(SAVE_PATH, 'inventory', huc, f'{gauge_id}.zarr'), mode = 'a')

# for idx, row in tqdm.tqdm(camels_graph.iterrows(), total=len(camels_graph)):
#     huc, gauge_id = row['huc_02'], row.name
#     compile_GLEAM4_zarr(huc, gauge_id)

with Parallel(n_jobs=8, verbose=10) as parallel:
    parallel(
        delayed(compile_GLEAM4_zarr)(row['huc_02'], row.name) for idx, row in tqdm.tqdm(camels_graph.iterrows(), total=len(camels_graph))
    )

  4%|▍         | 16/369 [00:11<04:11,  1.40it/s][Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:   17.8s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:   47.2s
100%|██████████| 369/369 [04:59<00:00,  1.23it/s]
[Parallel(n_jobs=8)]: Done 369 out of 369 | elapsed:  5.1min finished


# Daymet

In [11]:
def compile_Daymet_zarr(huc, gauge_id):
    warnings.filterwarnings('ignore')

    data_START_DATE = pd.Timestamp('1998-01-01')
    data_END_DATE = pd.Timestamp('2020-12-31')
    
    catmt = xr.open_zarr(os.path.join(SAVE_PATH, 'inventory', huc, f'{gauge_id}.zarr'))
    var_names = ['prcp', 'srad', 'tmax', 'tmin', 'vp', 'swe', 'dayl']
    for var_name in var_names:
        catmt[f'dynamic_Daymet_{var_name}'] = xr.DataArray(
            np.zeros((len(catmt.time), len(catmt.idx)), dtype = np.float32)*np.nan, 
            dims = ['time', 'idx'], 
            coords = {'time': catmt.time, 'idx': catmt.idx}
        )
        data = pd.read_csv(os.path.join(SAVE_PATH, 'graph_features', huc, gauge_id, 'dynamic', 'Daymet', f"{var_name}.csv"), index_col = 0, parse_dates = True)
        data = data.loc[data_START_DATE:data_END_DATE]
        for idx in data.columns:
            catmt[f'dynamic_Daymet_{var_name}'].loc[dict(time=slice(data_START_DATE, data_END_DATE), idx=int(idx))] = data[idx].values
        del data, idx

    catmt.to_zarr(os.path.join(SAVE_PATH, 'inventory', huc, f'{gauge_id}.zarr'), mode = 'a')

# for idx, row in tqdm.tqdm(camels_graph.iterrows(), total=len(camels_graph)):
#     huc, gauge_id = row['huc_02'], row.name
#     compile_Daymet_zarr(huc, gauge_id)

with Parallel(n_jobs=8, verbose=10) as parallel:
    parallel(
        delayed(compile_Daymet_zarr)(row['huc_02'], row.name) for idx, row in tqdm.tqdm(camels_graph.iterrows(), total=len(camels_graph))
    )

  4%|▍         | 16/369 [00:15<05:45,  1.02it/s][Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:   23.5s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:  1.1min
100%|██████████| 369/369 [06:58<00:00,  1.13s/it]
[Parallel(n_jobs=8)]: Done 369 out of 369 | elapsed:  7.0min finished


# GloFAS

In [12]:
def compile_GloFAS_dynamic_zarr(huc, gauge_id):
    warnings.filterwarnings('ignore')

    data_START_DATE = pd.Timestamp('1998-01-01')
    data_END_DATE = pd.Timestamp('2020-12-31')
    
    catmt = xr.open_zarr(os.path.join(SAVE_PATH, 'inventory', huc, f'{gauge_id}.zarr'))
    var_names = ['discharge_mm', 'runoff_water_equivalent', 'snow_depth_water_equivalent', 'soil_wetness_index']
    for var_name in var_names:
        catmt[f'dynamic_GloFAS_{var_name}'] = xr.DataArray(
            np.zeros((len(catmt.time), len(catmt.idx)), dtype = np.float32)*np.nan, 
            dims = ['time', 'idx'], 
            coords = {'time': catmt.time, 'idx': catmt.idx}
        )
        data = pd.read_csv(os.path.join(SAVE_PATH, 'graph_features', huc, gauge_id, 'dynamic', 'GloFAS', f"{var_name}.csv"), index_col = 0, parse_dates = True)
        data = data.loc[data_START_DATE:data_END_DATE]
        for idx in data.columns:
            catmt[f'dynamic_GloFAS_{var_name}'].loc[dict(time=slice(data_START_DATE, data_END_DATE), idx=int(idx))] = data[idx].values
        del data, idx

    catmt.to_zarr(os.path.join(SAVE_PATH, 'inventory', huc, f'{gauge_id}.zarr'), mode = 'a')

# for idx, row in tqdm.tqdm(camels_graph.iterrows(), total=len(camels_graph)):
#     huc, gauge_id = row['huc_02'], row.name
#     compile_GloFAS_dynamic_zarr(huc, gauge_id)

with Parallel(n_jobs=8, verbose=10) as parallel:
    parallel(
        delayed(compile_GloFAS_dynamic_zarr)(row['huc_02'], row.name) for idx, row in tqdm.tqdm(camels_graph.iterrows(), total=len(camels_graph))
    )

  4%|▍         | 16/369 [00:12<04:33,  1.29it/s][Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:   13.7s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:  1.1min
100%|██████████| 369/369 [06:39<00:00,  1.08s/it]
[Parallel(n_jobs=8)]: Done 369 out of 369 | elapsed:  6.7min finished


In [13]:
def compile_GloFAS_static_zarr(huc, gauge_id):
    catmt = xr.open_zarr(os.path.join(SAVE_PATH, 'inventory', huc, f'{gauge_id}.zarr'))
    var_names = ['chanbnkf', 'chanflpn', 'changrad', 'chanlength', 'chanman', 'chans', 'chanbw', 'fracforest', 'fracirrigated', 'fracrice', 'fracsealed', 'fracwater', 'fracother', 'cellarea_km2']
    for var_name in var_names:
        catmt[f'static_GloFAS_{var_name}'] = xr.DataArray(
            np.zeros((len(catmt.idx)), dtype = np.float32)*np.nan, 
            dims = ['idx'], 
            coords = {'idx': catmt.idx}
        )
        data = pd.read_csv(os.path.join(SAVE_PATH, 'graph_features', huc, gauge_id, 'static', 'GloFAS', f"{var_name}.csv"), index_col = 0, parse_dates = True)
        for idx in data.columns:
            catmt[f'static_GloFAS_{var_name}'].loc[dict(idx = int(idx))] = data[idx].values[0]
        del data, idx
    catmt.to_zarr(os.path.join(SAVE_PATH, 'inventory', huc, f'{gauge_id}.zarr'), mode = 'a')

# for idx, row in tqdm.tqdm(camels_graph.iterrows(), total=len(camels_graph)):
#     huc, gauge_id = row['huc_02'], row.name
#     compile_GloFAS_static_zarr(huc, gauge_id)

with Parallel(n_jobs=8, verbose=10) as parallel:
    parallel(
        delayed(compile_GloFAS_static_zarr)(row['huc_02'], row.name) for idx, row in tqdm.tqdm(camels_graph.iterrows(), total=len(camels_graph))
    )

  4%|▍         | 16/369 [00:13<04:59,  1.18it/s][Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:   13.8s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:  1.0min
100%|██████████| 369/369 [05:45<00:00,  1.07it/s]
[Parallel(n_jobs=8)]: Done 369 out of 369 | elapsed:  5.8min finished


# HWSD

In [ ]:
def compile_HWSD_static_zarr(huc, gauge_id):
    catmt = xr.open_zarr(os.path.join(SAVE_PATH, 'inventory', huc, f'{gauge_id}.zarr'))
    var_names = ['S_CLAY', 'S_GRAVEL', 'S_SAND', 'S_SILT', 'T_CLAY', 'T_GRAVEL', 'T_SAND', 'T_SILT']
    for var_name in var_names:
        catmt[f'static_HWSD_{var_name}'] = xr.DataArray(
            np.zeros((len(catmt.idx)), dtype = np.float32)*np.nan, 
            dims = ['idx'], 
            coords = {'idx': catmt.idx}
        )
        data = pd.read_csv(os.path.join(SAVE_PATH, 'graph_features', huc, gauge_id, 'static', 'HWSD', f"{var_name}.csv"), index_col = 0, parse_dates = True)
        for idx in data.columns:
            catmt[f'static_HWSD_{var_name}'].loc[dict(idx = int(idx))] = data[idx].values[0]
        del data, idx
    catmt.to_zarr(os.path.join(SAVE_PATH, 'inventory', huc, f'{gauge_id}.zarr'), mode = 'a')

# for idx, row in tqdm.tqdm(camels_graph.iterrows(), total=len(camels_graph)):
#     huc, gauge_id = row['huc_02'], row.name
#     compile_HWSD_static_zarr(huc, gauge_id)

with Parallel(n_jobs=8, verbose=10) as parallel:
    parallel(
        delayed(compile_HWSD_static_zarr)(row['huc_02'], row.name) for idx, row in tqdm.tqdm(camels_graph.iterrows(), total=len(camels_graph))
    )

# Solar Insolation

In [15]:
def compile_solar_insolation_zarr(huc, gauge_id):
    warnings.filterwarnings('ignore')

    data_START_DATE = pd.Timestamp('1998-01-01')
    data_END_DATE = pd.Timestamp('2020-12-31')

    catmt = xr.open_zarr(os.path.join(SAVE_PATH, 'inventory', huc, f'{gauge_id}.zarr'))

    catmt[f'encoding_solar_insolation'] = xr.DataArray(
        np.zeros((len(catmt.time), len(catmt.idx)), dtype = np.float32)*np.nan, 
        dims = ['time', 'idx'], 
        coords = {'time': catmt.time, 'idx': catmt.idx}
    )
    data = pd.read_csv(os.path.join(SAVE_PATH, 'graph_features', huc, gauge_id, "solar_insolation.csv"), index_col = 0, parse_dates = True)
    data = data.loc[data_START_DATE:data_END_DATE]
    for idx in data.columns:
        catmt[f'encoding_solar_insolation'].loc[dict(time=slice(data_START_DATE, data_END_DATE), idx=int(idx))] = data[idx].values
    del data, idx

    catmt.to_zarr(os.path.join(SAVE_PATH, 'inventory', huc, f'{gauge_id}.zarr'), mode = 'a')

# for idx, row in tqdm.tqdm(camels_graph.iterrows(), total=len(camels_graph)):
#     huc, gauge_id = row['huc_02'], row.name
#     compile_solar_insolation_zarr(huc, gauge_id)

with Parallel(n_jobs=8, verbose=10) as parallel:
    parallel(
        delayed(compile_solar_insolation_zarr)(row['huc_02'], row.name) for idx, row in tqdm.tqdm(camels_graph.iterrows(), total=len(camels_graph))
    )

  0%|          | 0/369 [00:00<?, ?it/s]

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
  4%|▍         | 16/369 [00:10<03:59,  1.47it/s][Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:   11.6s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:   58.5s
100%|██████████| 369/369 [06:17<00:00,  1.02s/it]
[Parallel(n_jobs=8)]: Done 369 out of 369 | elapsed:  6.3min finished


# Time Encodings

In [16]:
def compile_time_encodings_zarr(huc, gauge_id):
    warnings.filterwarnings('ignore')

    data_START_DATE = pd.Timestamp('1998-01-01')
    data_END_DATE = pd.Timestamp('2020-12-31')

    catmt = xr.open_zarr(os.path.join(SAVE_PATH, 'inventory', huc, f'{gauge_id}.zarr'))
    var_names = ['sine_month', 'sine_weekofyear', 'sine_dayofyear']
    data_all_vars = pd.read_csv(os.path.join(SAVE_PATH, 'graph_features', huc, gauge_id, "time_encodings.csv"), index_col = 0, parse_dates = True)
    data_all_vars = data_all_vars.loc[data_START_DATE:data_END_DATE]
    for var_name in var_names:
        data = data_all_vars[var_name].values
        data = data.astype(np.float32)
        catmt[f'encoding_{var_name}'] = xr.DataArray(
            np.zeros((len(catmt.time)), dtype = np.float32)*np.nan,
            dims = ['time'], 
            coords = {'time': catmt.time}
        )
        catmt[f'encoding_{var_name}'].loc[dict(time=slice(data_START_DATE, data_END_DATE))] = data
        del data

    catmt.to_zarr(os.path.join(SAVE_PATH, 'inventory', huc, f'{gauge_id}.zarr'), mode = 'a')

# for idx, row in tqdm.tqdm(camels_graph.iterrows(), total=len(camels_graph)):
#     huc, gauge_id = row['huc_02'], row.name
#     compile_time_encodings_zarr(huc, gauge_id)

with Parallel(n_jobs=8, verbose=10) as parallel:
    parallel(
        delayed(compile_time_encodings_zarr)(row['huc_02'], row.name) for idx, row in tqdm.tqdm(camels_graph.iterrows(), total=len(camels_graph))
    )

  4%|▍         | 16/369 [00:27<10:15,  1.74s/it][Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:   29.9s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:  1.1min
100%|██████████| 369/369 [06:02<00:00,  1.02it/s]
[Parallel(n_jobs=8)]: Done 369 out of 369 | elapsed:  6.1min finished


# Spatial Encodings

In [ ]:
def compile_spatial_encoding_zarr(huc, gauge_id):
    catmt = xr.open_zarr(os.path.join(SAVE_PATH, 'inventory', huc, f'{gauge_id}.zarr'))

    catmt[f'encoding_transformed_longitude'] = xr.DataArray(
        np.zeros((len(catmt.idx)), dtype = np.float32)*np.nan, 
        dims = ['idx'], 
        coords = {'idx': catmt.idx}
    )
    catmt[f'encoding_transformed_latitude'] = xr.DataArray(
        np.zeros((len(catmt.idx)), dtype = np.float32)*np.nan, 
        dims = ['idx'], 
        coords = {'idx': catmt.idx}
    )

    # lon: -180 to 180; lat: -60 to 90
    lon_transform = lambda x: np.sin(2 * np.pi * (x+180) / 360)
    lat_transform = lambda x: (x - (-60))/(90 - (-60))

    nodes_coords = pd.read_csv(os.path.join(SAVE_PATH, 'graph_files', huc, gauge_id, 'nodes_coords.csv'), index_col = 0)
    for node_idx, node_row in nodes_coords.iterrows():
        lat, lon = node_row['lat'], node_row['lon']
        lon_transformed = lon_transform(lon)
        lat_transformed = lat_transform(lat)
        catmt['encoding_transformed_longitude'].loc[dict(idx = node_idx)] = lon_transformed
        catmt['encoding_transformed_latitude'].loc[dict(idx = node_idx)] = lat_transformed

    catmt.to_zarr(os.path.join(SAVE_PATH, 'inventory', huc, f'{gauge_id}.zarr'), mode = 'a')

# for idx, row in tqdm.tqdm(camels_graph.iterrows(), total=len(camels_graph)):
#     huc, gauge_id = row['huc_02'], row.name
#     compile_spatial_encoding_zarr(huc, gauge_id)

with Parallel(n_jobs=8, verbose=10) as parallel:
    parallel(
        delayed(compile_spatial_encoding_zarr)(row['huc_02'], row.name) for idx, row in tqdm.tqdm(camels_graph.iterrows(), total=len(camels_graph))
    )

# uparea

In [ ]:
def compile_LDD_uparea_zarr(huc, gauge_id):
    catmt = xr.open_zarr(os.path.join(SAVE_PATH, 'inventory', huc, f'{gauge_id}.zarr'))
    var_name = 'uparea'
    catmt[f'static_uparea'] = xr.DataArray(
        np.zeros((len(catmt.idx)), dtype = np.float32)*np.nan, 
        dims = ['idx'], 
        coords = {'idx': catmt.idx}
    )
    data = pd.read_csv(os.path.join(SAVE_PATH, 'graph_features', huc, gauge_id, 'uparea.csv'), index_col = 0, parse_dates = True)
    for idx in data.columns:
        catmt[f'static_uparea'].loc[dict(idx = int(idx))] = data[idx].values[0]
    del data, idx
    catmt.to_zarr(os.path.join(SAVE_PATH, 'inventory', huc, f'{gauge_id}.zarr'), mode = 'a')

# for idx, row in tqdm.tqdm(camels_graph.iterrows(), total=len(camels_graph)):
#     huc, gauge_id = row['huc_02'], row.name
#     compile_LDD_uparea_zarr(huc, gauge_id)

with Parallel(n_jobs=8, verbose=10) as parallel:
    parallel(
        delayed(compile_LDD_uparea_zarr)(row['huc_02'], row.name) for idx, row in tqdm.tqdm(camels_graph.iterrows(), total=len(camels_graph))
    )

# MERIT-Hydro

In [ ]:
def compile_terrain_static_zarr(huc, gauge_id):
    catmt = xr.open_zarr(os.path.join(SAVE_PATH, 'inventory', huc, f'{gauge_id}.zarr'))
    var_names = ['elv', 'slope_percentage', 'slope_riserun', 'slope_degrees', 'slope_radians', 'aspect', 'curvature', 'planform_curvature', 'profile_curvature', 'upa', 'wth']
    metrics = ['mean', 'std', '50%']
    for var_name in var_names:
        data = pd.read_csv(os.path.join(SAVE_PATH, 'graph_features', huc, gauge_id, 'static', 'MERIT-Hydro', f"{var_name}.csv"), index_col = 0, parse_dates = True)
        for metric in metrics:
            catmt[f'static_terrain_{var_name}_{metric}'] = xr.DataArray(
                np.zeros((len(catmt.idx)), dtype = np.float32)*np.nan, 
                dims = ['idx'], 
                coords = {'idx': catmt.idx}
            )
            for idx in data.columns:
                catmt[f'static_terrain_{var_name}_{metric}'].loc[dict(idx = int(idx))] = data.loc[metric, idx]
        del data, idx
    catmt.to_zarr(os.path.join(SAVE_PATH, 'inventory', huc, f'{gauge_id}.zarr'), mode = 'a')

# for idx, row in tqdm.tqdm(camels_graph.iterrows(), total=len(camels_graph)):
#     huc, gauge_id = row['huc_02'], row.name
#     compile_terrain_static_zarr(huc, gauge_id)

with Parallel(n_jobs=8, verbose=10) as parallel:
    parallel(
        delayed(compile_terrain_static_zarr)(row['huc_02'], row.name) for idx, row in tqdm.tqdm(camels_graph.iterrows(), total=len(camels_graph))
    )

# USGS

In [68]:
def compile_USGS_outlet_zarr(huc, gauge_id):
    warnings.filterwarnings('ignore')

    data_START_DATE = pd.Timestamp('1998-01-01')
    data_END_DATE = pd.Timestamp('2020-12-31')

    catmt = xr.open_zarr(os.path.join(SAVE_PATH, 'inventory', huc, f'{gauge_id}.zarr'))
    var_name = 'Q_mm'
    data = pd.read_csv(os.path.join(SAVE_PATH, 'graph_features', huc, gauge_id, "USGS.csv"), index_col = 0, parse_dates = True)
    data = data.loc[data_START_DATE:data_END_DATE]
    data = data[var_name].values
    data = data.astype(np.float32)
    catmt[f'outlet_USGS_{var_name}'] = xr.DataArray(
        np.zeros((len(catmt.time)), dtype = np.float32)*np.nan,
        dims = ['time'], 
        coords = {'time': catmt.time}
    )
    catmt[f'outlet_USGS_{var_name}'].loc[dict(time=slice(data_START_DATE, data_END_DATE))] = data
    del data

    catmt.to_zarr(os.path.join(SAVE_PATH, 'inventory', huc, f'{gauge_id}.zarr'), mode = 'a')

# for idx, row in tqdm.tqdm(camels_graph.iterrows(), total=len(camels_graph)):
#     huc, gauge_id = row['huc_02'], row.name
#     compile_USGS_outlet_zarr(huc, gauge_id)

with Parallel(n_jobs=8, verbose=10) as parallel:
    parallel(
        delayed(compile_USGS_outlet_zarr)(row['huc_02'], row.name) for idx, row in tqdm.tqdm(camels_graph.iterrows(), total=len(camels_graph))
    )

 62%|██████▏   | 16/26 [00:06<00:04,  2.45it/s][Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    7.5s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:   13.2s
100%|██████████| 26/26 [00:13<00:00,  1.98it/s]
[Parallel(n_jobs=8)]: Done  14 out of  26 | elapsed:   14.2s remaining:   12.2s
[Parallel(n_jobs=8)]: Done  17 out of  26 | elapsed:   15.2s remaining:    8.0s
[Parallel(n_jobs=8)]: Done  20 out of  26 | elapsed:   16.6s remaining:    5.0s
[Parallel(n_jobs=8)]: Done  23 out of  26 | elapsed:   16.9s remaining:    2.2s
[Parallel(n_jobs=8)]: Done  26 out of  26 | elapsed:   18.1s finished


# ERA5-Land

In [69]:
def compile_ERA5Land_zarr(huc, gauge_id):

    warnings.filterwarnings('ignore')

    data_START_DATE = pd.Timestamp('1998-01-01')
    data_END_DATE = pd.Timestamp('2019-12-31')

    catmt = xr.open_zarr(os.path.join(SAVE_PATH, 'inventory', huc, f'{gauge_id}.zarr'))
    var_names = [
        'total_precipitation_sum',
        'temperature_2m_min',
        'temperature_2m_max',
        'total_evaporation_sum',
        'potential_evaporation_sum',
        'surface_net_solar_radiation_sum',
        'surface_net_thermal_radiation_sum',
        'surface_pressure',
        'u_component_of_wind_10m',
        'v_component_of_wind_10m',
        'snowfall_sum',
        'snowmelt_sum',
        'snow_depth',
        'snow_cover',
        'dewpoint_temperature_2m_min',
        'dewpoint_temperature_2m_max',
        'leaf_area_index_high_vegetation',
        'leaf_area_index_low_vegetation',
        'runoff_sum',
        'surface_runoff_sum',
        'sub_surface_runoff_sum',
        'volumetric_soil_water_layer_1',
        'volumetric_soil_water_layer_2',
        'volumetric_soil_water_layer_3',
        'volumetric_soil_water_layer_4',
    ]
    for var_name in var_names:
        catmt[f'dynamic_ERA5-Land_{var_name}'] = xr.DataArray(
            np.zeros((len(catmt.time), len(catmt.idx)), dtype = np.float32)*np.nan, 
            dims = ['time', 'idx'], 
            coords = {'time': catmt.time, 'idx': catmt.idx}
        )
        data = pd.read_csv(os.path.join(SAVE_PATH, 'graph_features', huc, gauge_id, 'dynamic', 'ERA5-Land', f"{var_name}.csv"), index_col = 0, parse_dates = True)
        data = data.loc[data_START_DATE:data_END_DATE]
        for idx in data.columns:
            catmt[f'dynamic_ERA5-Land_{var_name}'].loc[dict(time=slice(data_START_DATE, data_END_DATE), idx=int(idx))] = data[idx].values
        del data, idx

    catmt.to_zarr(os.path.join(SAVE_PATH, 'inventory', huc, f'{gauge_id}.zarr'), mode = 'a')

# for idx, row in tqdm.tqdm(camels_graph.iterrows(), total=len(camels_graph)):
#     huc, gauge_id = row['huc_02'], row.name
#     compile_ERA5Land_zarr(huc, gauge_id)

with Parallel(n_jobs=8, verbose=10) as parallel:
    parallel(
        delayed(compile_ERA5Land_zarr)(row['huc_02'], row.name) for idx, row in tqdm.tqdm(camels_graph.iterrows(), total=len(camels_graph))
    )

 62%|██████▏   | 16/26 [00:10<00:06,  1.59it/s][Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:   11.3s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:   25.9s
100%|██████████| 26/26 [00:25<00:00,  1.00it/s]
[Parallel(n_jobs=8)]: Done  14 out of  26 | elapsed:   33.9s remaining:   29.1s
[Parallel(n_jobs=8)]: Done  17 out of  26 | elapsed:   37.0s remaining:   19.6s
[Parallel(n_jobs=8)]: Done  20 out of  26 | elapsed:   38.7s remaining:   11.6s
[Parallel(n_jobs=8)]: Done  23 out of  26 | elapsed:   40.3s remaining:    5.3s
[Parallel(n_jobs=8)]: Done  26 out of  26 | elapsed:   42.9s finished


# GPM

In [80]:
def compile_GPM_zarr(huc, gauge_id):

    warnings.filterwarnings('ignore')

    data_START_DATE = pd.Timestamp('1999-01-01')
    data_END_DATE = pd.Timestamp('2020-12-31')

    catmt = xr.open_zarr(os.path.join(SAVE_PATH, 'inventory', huc, f'{gauge_id}.zarr'))
    sources = [
        'Early_Run',
        'Late_Run',
        'Final_Run'
    ]
    for source in sources:
        catmt[f'dynamic_GPM_{source}'] = xr.DataArray(
            np.zeros((len(catmt.time), len(catmt.idx)), dtype = np.float32)*np.nan, 
            dims = ['time', 'idx'], 
            coords = {'time': catmt.time, 'idx': catmt.idx}
        )
        # data = pd.read_csv(os.path.join(SAVE_PATH, 'graph_features', huc, gauge_id, 'dynamic', 'GPM', source, f"*.csv"), index_col = 0, parse_dates = True)
        data_files = sorted(glob.glob(os.path.join(SAVE_PATH, 'graph_features', huc, gauge_id, 'dynamic', 'GPM', source, "*.csv")))
        data = pd.concat([pd.read_csv(file, index_col=0, parse_dates=True) for file in data_files], axis=0)
        data = data.loc[data_START_DATE:data_END_DATE]
        for idx in data.columns:
            catmt[f'dynamic_GPM_{source}'].loc[dict(time=slice(data_START_DATE, data_END_DATE), idx=int(idx))] = data[idx].values
        del data, idx

    catmt.to_zarr(os.path.join(SAVE_PATH, 'inventory', huc, f'{gauge_id}.zarr'), mode = 'a')

# for idx, row in tqdm.tqdm(camels_graph.iterrows(), total=len(camels_graph)):
#     huc, gauge_id = row['huc_02'], row.name
#     compile_GPM_zarr(huc, gauge_id)

with Parallel(n_jobs=8, verbose=10) as parallel:
    parallel(
        delayed(compile_GPM_zarr)(row['huc_02'], row.name) for idx, row in tqdm.tqdm(camels_graph.iterrows(), total=len(camels_graph))
    )

 62%|██████▏   | 16/26 [00:15<00:11,  1.10s/it][Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:   16.8s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:   28.1s
100%|██████████| 26/26 [00:28<00:00,  1.08s/it]
[Parallel(n_jobs=8)]: Done  14 out of  26 | elapsed:   29.3s remaining:   25.1s
[Parallel(n_jobs=8)]: Done  17 out of  26 | elapsed:   34.5s remaining:   18.3s
[Parallel(n_jobs=8)]: Done  20 out of  26 | elapsed:   34.8s remaining:   10.5s
[Parallel(n_jobs=8)]: Done  23 out of  26 | elapsed:   34.9s remaining:    4.6s
[Parallel(n_jobs=8)]: Done  26 out of  26 | elapsed:   38.6s finished


# ECMWF_HRES

In [83]:
def compile_ECMWF_HRES_zarr(huc, gauge_id):

    warnings.filterwarnings('ignore')

    data_START_DATE = pd.Timestamp('2016-01-01')
    data_END_DATE = pd.Timestamp('2020-12-31')

    catmt = xr.open_zarr(os.path.join(SAVE_PATH, 'inventory', huc, f'{gauge_id}.zarr'))
    var_names = [
        '2m_temperature',
        '10m_u_component_of_wind',
        '10m_v_component_of_wind',
        'total_precipitation_24hr',
        'surface_pressure',
    ]
    leads = [24*x for x in range(1, 10+1)]
    catmt = catmt.assign_coords({'lead': leads})
    for var_name in var_names:
        catmt[f'dynamic_HRES_{var_name}'] = xr.DataArray(
            np.zeros((len(catmt.time), len(catmt.idx), len(catmt.lead)), dtype=np.float32) * np.nan, 
            dims=['time', 'idx', 'lead'], 
            coords={'time': catmt.time, 'idx': catmt.idx, 'lead': catmt.lead}
        )
        for lead in leads:
            data = pd.read_csv(os.path.join(SAVE_PATH, 'graph_features', huc, gauge_id, 'dynamic', 'ECMWF_HRES', var_name, f"{lead}hrs.csv"), index_col = 0, parse_dates = True)
            data = data.loc[data_START_DATE:data_END_DATE]
            for idx in data.columns:
                catmt[f'dynamic_HRES_{var_name}'].loc[dict(time=slice(data_START_DATE, data_END_DATE), idx=int(idx), lead=lead)] = data[idx].values
            del data, idx

    catmt.to_zarr(os.path.join(SAVE_PATH, 'inventory', huc, f'{gauge_id}.zarr'), mode = 'a')

# for idx, row in tqdm.tqdm(camels_graph.iterrows(), total=len(camels_graph)):
#     huc, gauge_id = row['huc_02'], row.name
#     compile_ECMWF_HRES_zarr(huc, gauge_id)

with Parallel(n_jobs=8, verbose=10) as parallel:
    parallel(
        delayed(compile_ECMWF_HRES_zarr)(row['huc_02'], row.name) for idx, row in tqdm.tqdm(camels_graph.iterrows(), total=len(camels_graph))
    )

 62%|██████▏   | 16/26 [00:14<00:10,  1.08s/it][Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:   18.0s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:   32.5s
100%|██████████| 26/26 [00:32<00:00,  1.25s/it]
[Parallel(n_jobs=8)]: Done  14 out of  26 | elapsed:   38.7s remaining:   33.2s
[Parallel(n_jobs=8)]: Done  17 out of  26 | elapsed:   43.2s remaining:   22.9s
[Parallel(n_jobs=8)]: Done  20 out of  26 | elapsed:   45.9s remaining:   13.8s
[Parallel(n_jobs=8)]: Done  23 out of  26 | elapsed:   46.4s remaining:    6.1s
[Parallel(n_jobs=8)]: Done  26 out of  26 | elapsed:   51.1s finished


# Check sample

In [6]:
row = camels_graph.iloc[0]
huc, gauge_id = row['huc_02'], row.name
catmt = xr.open_zarr(os.path.join(SAVE_PATH, 'inventory', huc, f'{gauge_id}.zarr'))
catmt[sorted(catmt.data_vars)]

<xarray.Dataset> Size: 5GB
Dimensions:                                              (time: 9125,
                                                          idx: 1158, lead: 10,
                                                          lat: 29, lon: 87)
Coordinates:
  * idx                                                  (idx) int32 5kB 0 .....
  * lon                                                  (lon) float32 348B -...
  * time                                                 (time) datetime64[ns] 73kB ...
  * lead                                                 (lead) int32 40B 24 ...
  * lat                                                  (lat) float32 116B 4...
Data variables: (12/138)
    dynamic_Daymet_dayl                                  (time, idx) float32 42MB dask.array<chunksize=(1141, 145), meta=np.ndarray>
    dynamic_Daymet_prcp                                  (time, idx) float32 42MB dask.array<chunksize=(1141, 145), meta=np.ndarray>
    dynamic_Daymet_srad                                  (time, idx) float32 42MB dask.array<chunksize=(1141, 145), meta=np.ndarray>
    dynamic_Daymet_swe                                   (time, idx) float32 42MB dask.array<chunksize=(1141, 145), meta=np.ndarray>
    dynamic_Daymet_tmax                                  (time, idx) float32 42MB dask.array<chunksize=(1141, 145), meta=np.ndarray>
    dynamic_Daymet_tmin                                  (time, idx) float32 42MB dask.array<chunksize=(1141, 145), meta=np.ndarray>
    ...                                                   ...
    static_terrain_upa_mean                              (idx) float32 5kB dask.array<chunksize=(1158,), meta=np.ndarray>
    static_terrain_upa_std                               (idx) float32 5kB dask.array<chunksize=(1158,), meta=np.ndarray>
    static_terrain_wth_50%                               (idx) float32 5kB dask.array<chunksize=(1158,), meta=np.ndarray>
    static_terrain_wth_mean                              (idx) float32 5kB dask.array<chunksize=(1158,), meta=np.ndarray>
    static_terrain_wth_std                               (idx) float32 5kB dask.array<chunksize=(1158,), meta=np.ndarray>
    static_uparea                                        (idx) float32 5kB dask.array<chunksize=(1158,), meta=np.ndarray>
Attributes:
    watershed:            CAMELS-US/HUC:10/GaugeID:06452000
    gauge_idx:            [0]
    resolution(degrees):  0.05
    resolution(arcmins):  3
    edges_idx:            [[1157, 1155], [1156, 1154], [1155, 1151], [1154, 1...
    drainage_area_km2:    25817.78